In [14]:
import os
import numpy as np
import matplotlib as mpl
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

DATASET_DIRECTORY = r"C:\Users\user\Desktop\Face-Mask-Detection-Using-Hardware\dataset"
TYPES = ["with_mask", "without_mask"]

# All the Images Array will be appended in data list, whereas, the label of images(with_mask or without_mask) will be appended in labels
data = []
labels = []

print("Loading Images present our dataset!")

for dir_type in TYPES:
    path = os.path.join(DATASET_DIRECTORY,dir_type)
    print(path);
    #listdir(path) function will list down all the diretories in the path specified
    for img in os.listdir(path):
        image_path = os.path.join(path,img)
        image = load_img(image_path, target_size=(244,244))
        image = img_to_array(image)
        image = preprocess_input(image)
        
        data.append(image)
        labels.append(dir_type)

# We've got the data as numerical values, however, the labels are still alphabetical values
# Thus we will convert them into categorical variables using LabelBinarizer from sklearn library
        
binarizer = LabelBinarizer()
labels = binarizer.fit_transform(labels)
labels = to_categorical(labels)

# Now, to apply deep learning models, we will convert the data and labels list into numpy arrays
data = np.array(data, dtype = "float32")
labels = np.array(labels)

# Splitting Testing and Training data using test_train_split() function
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, stratify=labels, random_state=42)

print("Data Preprocessing Completed! Images loaded and splitted into training and testing data!")



Loading Images present our dataset!
C:\Users\user\Desktop\Face-Mask-Detection-Using-Hardware\dataset\with_mask
C:\Users\user\Desktop\Face-Mask-Detection-Using-Hardware\dataset\without_mask
Data Preprocessing Completed! Images loaded and splitted into training and testing data!
